### Este repo es una copia, el original lo puedes encontrar en:

https://github.com/ayushdixit487/Uber-Data-Analysis-Project-in-Pyspark

https://medium.com/towards-data-engineering/uber-data-analysis-project-in-pyspark-e105a445fc3e

# Uber-Data-Analysis-Project-in-Pyspark
<img width="374" alt="image" src="https://user-images.githubusercontent.com/25612446/219968335-5e40c842-56cd-4f87-97b8-bb304a8a4c69.png">

This data project can be used as a take-home assignment to learn Pyspark and Data Engineering.

## Insights from City Supply and Demand Data

## Data Description

To answer the question, use the dataset from the file dataset.csv. For example, consider a row from this dataset:


![image](https://user-images.githubusercontent.com/25612446/219965433-956a1dc0-2acf-4d0d-b1cb-40723249d349.png)


This means that during the hour beginning at 4pm (hour 16), on September 10th, 2012, 11 people opened the Uber app (Eyeballs). 2 of them did not see any car (Zeroes) and 4 of them requested a car (Requests). Of the 4 requests, only 3 complete trips actually resulted (Completed Trips). During this time, there were a total of 6 drivers who logged in (Unique Drivers).

## Actividad

Usando el dataset, responder a las siguientes preguntas


⚡ ¿Qué fecha tuvo la mayoría de los viajes completados durante el período de dos semanas?

⚡ ¿Cuál fue el número más alto de viajes completados dentro de un período de 24 horas?

⚡ ¿Qué hora del día tuvo más solicitudes durante el período de dos semanas?

⚡ ¿Qué porcentaje de todos los ceros durante el período de dos semanas ocurrieron durante el fin de semana (viernes a las 5 pm hasta el domingo a las 3 am)? Consejo: El valor de la hora local es el inicio de la hora (por ejemplo, 15 es la hora de 3:00 pm a 4:00 pm).

⚡ ¿Cuál es la proporción promedio ponderada de viajes completados por conductor durante el período de dos semanas? Consejo: "Promedio ponderado" significa que tu respuesta debe tener en cuenta el volumen total de viajes en cada hora para determinar el número más preciso en todo el período.

⚡ Al redactar un horario de conductor en términos de turnos de 8 horas, ¿cuándo son las 8 horas consecutivas más ocupadas durante el período de dos semanas en términos de solicitudes únicas? Un nuevo turno comienza cada 8 horas. Supón que un conductor trabajará el mismo turno cada día.

⚡ Verdadero o Falso: ¿La oferta de conductores siempre aumenta cuando aumenta la demanda durante el período de dos semanas? Consejo: Visualiza los datos para confirmar tu respuesta si es necesario.

⚡ ¿En qué período de 72 horas es la proporción de Ceros a Miradas más alta?

⚡ Si pudieras agregar 5 conductores a cualquier hora única de cada día durante el período de dos semanas, ¿a qué hora deberías agregarlos? Pista: Considera tanto las miradas de los pasajeros como la oferta de conductores al elegir.

⚡ Al observar los datos de todas las dos semanas, ¿qué hora podría tener más sentido considerar como un verdadero "fin de día" en lugar de la medianoche? (es decir, ¿cuándo la oferta y la demanda están en sus mínimos naturales?) Consejo: Visualiza los datos para confirmar tu respuesta si es necesario.


Fuente: https://medium.com/towards-data-engineering/uber-data-analysis-project-in-pyspark-e105a445fc3e





In [1]:
# Import the necessary modules
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

# Create a SparkSession
spark = SparkSession.builder \
   .appName("My App") \
   .config("spark.driver.bindAddress", "172.17.0.1",)\
   .config("spark.driver.port", "7077")\
   .getOrCreate()

your 131072x1 screen size is bogus. expect trouble
24/03/01 14:05:39 WARN Utils: Your hostname, ANDRES-SOTOH resolves to a loopback address: 127.0.1.1; using 172.20.226.32 instead (on interface eth0)
24/03/01 14:05:39 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/01 14:05:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/01 14:05:43 WARN Utils: Service 'sparkDriver' could not bind on port 7077. Attempting port 7078.
24/03/01 14:05:44 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [93]:
from pyspark.sql.functions import max
from pyspark.sql.functions import to_timestamp

spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")

# Read the data from CSV file
uber = spark.read.csv("dataset.csv", header=True, inferSchema=True, sep=';')


In [94]:
uber = uber.withColumn("datetime", to_timestamp("Time (Local)", "dd-MMM-yyyy HH:mm:ss"))


In [95]:
uber.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Time (Local): string (nullable = true)
 |-- Eyeballs: integer (nullable = true)
 |-- Zeroes: integer (nullable = true)
 |-- Completed Trips: integer (nullable = true)
 |-- Requests: integer (nullable = true)
 |-- Unique Drivers: integer (nullable = true)
 |-- datetime: timestamp (nullable = true)



In [96]:
uber.count()

336

In [97]:

# Group the data by date and sum the completed trips
completed_trips_by_date = uber.groupBy("Date").sum("Completed Trips")

# Find the date with the most completed trips
date_with_most_completed_trips = completed_trips_by_date \
    .orderBy("sum(Completed Trips)", ascending=False) \
    .select("Date") \
    .first()["Date"]

print(date_with_most_completed_trips)


22-Sep-12


In [98]:
uber.show()

+---------+--------------------+--------+------+---------------+--------+--------------+-------------------+
|     Date|        Time (Local)|Eyeballs|Zeroes|Completed Trips|Requests|Unique Drivers|           datetime|
+---------+--------------------+--------+------+---------------+--------+--------------+-------------------+
|10-Sep-12| 10-Sep-2012 7:00:00|       5|     0|              2|       2|             9|2012-09-10 07:00:00|
|10-Sep-12| 10-Sep-2012 8:00:00|       6|     0|              2|       2|            14|2012-09-10 08:00:00|
|10-Sep-12| 10-Sep-2012 9:00:00|       8|     3|              0|       0|            14|2012-09-10 09:00:00|
|10-Sep-12|10-Sep-2012 10:00:00|       9|     2|              0|       1|            14|2012-09-10 10:00:00|
|10-Sep-12|10-Sep-2012 11:00:00|      11|     1|              4|       4|            11|2012-09-10 11:00:00|
|10-Sep-12|10-Sep-2012 12:00:00|      12|     0|              2|       2|            11|2012-09-10 12:00:00|
|10-Sep-12|10-Sep-2

In [102]:
from pyspark.sql.functions import sum, window


# Group the data by 24-hour windows and sum the completed trips
completed_trips_by_window = uber \
    .groupBy(window("datetime", "24 hours")) \
    .agg(sum("Completed Trips").alias("Total Completed Trips")) \
    .orderBy("Total Completed Trips", ascending=False)


completed_trips_by_window.show()

+--------------------+---------------------+
|              window|Total Completed Trips|
+--------------------+---------------------+
|{2012-09-21 19:00...|                  256|
|{2012-09-22 19:00...|                  189|
|{2012-09-14 19:00...|                  181|
|{2012-09-15 19:00...|                  161|
|{2012-09-20 19:00...|                  130|
|{2012-09-11 19:00...|                   96|
|{2012-09-13 19:00...|                   63|
|{2012-09-16 19:00...|                   51|
|{2012-09-17 19:00...|                   47|
|{2012-09-19 19:00...|                   46|
|{2012-09-18 19:00...|                   42|
|{2012-09-12 19:00...|                   40|
|{2012-09-10 19:00...|                   24|
|{2012-09-09 19:00...|                   20|
|{2012-09-23 19:00...|                   19|
+--------------------+---------------------+



In [100]:

# Get the highest number of completed trips within a 24-hour period
highest_completed_trips_in_24_hours = completed_trips_by_window \
    .select("Total Completed Trips") \
    .first()["Total Completed Trips"]

print(highest_completed_trips_in_24_hours)


256


In [104]:
from pyspark.sql.functions import hour, sum

hourly_requests = uber.groupBy(hour("datetime").alias("hour")).agg(sum("Requests").alias("total_requests")).orderBy("total_requests", ascending=False)

most_requested_hour = hourly_requests.select("hour").first()[0]
print("The hour with the most requests is:", most_requested_hour)


The hour with the most requests is: 23
